# Inverted Pendulum on a Drone

In [1]:
import numpy as np
import sympy as sym
import matplotlib.pyplot as plt
from scipy import linalg
from scipy.interpolate import interp1d
from sympy.physics.mechanics import *

### Standard Inverted Pendulum Dynamics?
We could derive the dynamics for a standard inverted pendulum, but now the acceleration input is dependent on the angles of the base...

Define states

In [2]:
# components of position (meters)
o_x, o_y, o_z = sym.symbols('o_x, o_y, o_z')

# yaw, pitch, and roll angles (radians)
psi, theta, phi = sym.symbols('psi, theta, phi')

# components of linear velocity (meters / second)
v_x, v_y, v_z = sym.symbols('v_x, v_y, v_z')

# components of angular velocity (radians / second)
w_x, w_y, w_z = sym.symbols('w_x, w_y, w_z')

# states of the pendulum
alpha, alpha_dot = sym.symbols('alpha, alpha_dot')

Define inputs

In [3]:
# components of net rotor torque
tau_x, tau_y, tau_z = sym.symbols('tau_x, tau_y, tau_z')

# net rotor force
f_z = sym.symbols('f_z')

Define parameters

m_pen is the mass of the pendulum and l_pen is the length of the pendulum

In [4]:
m, J_x, J_y, J_z, g, m_pen, l_pen = sym.symbols('m, J_x, J_y, J_z, g, m_pen, l_pen')

In [5]:
# l_pen = 200e-3 # maybe m
# density_pen = 8000 # kg/m3
# diameter_pen = 1.2e-3 # m
# m_pen = np.pi * (diameter_pen)**2/4 * l_pen * diameter_pen

Create linear and angular velocity vectors (in coordinates of the body frame).

In [6]:
v_01in1 = sym.Matrix([v_x, v_y, v_z])
w_01in1 = sym.Matrix([w_x, w_y, w_z])

Create moment of inertia matrix (in coordinates of the body frame).

In [7]:
J_in1 = sym.diag(J_x, J_y, J_z)

## 2.2 Define kinematics of orientation

### 2.2.1 Rotation matrix in terms of yaw, pitch, roll angles

Define individual rotation matrices.

In [8]:
Rz = sym.Matrix([[sym.cos(psi), -sym.sin(psi), 0],
                 [sym.sin(psi), sym.cos(psi), 0],
                 [0, 0, 1]])

Ry = sym.Matrix([[sym.cos(theta), 0, sym.sin(theta)],
                 [0, 1, 0],
                 [-sym.sin(theta), 0, sym.cos(theta)]])

Rx = sym.Matrix([[1, 0, 0],
                 [0, sym.cos(phi), -sym.sin(phi)],
                 [0, sym.sin(phi), sym.cos(phi)]])

Apply sequential transformation to compute the rotation matrix that describes the orientation of the drone (i.e., of frame 1 in the coordinates of frame 0).

In [9]:
R_1in0 = Rz * Ry * Rx

In [10]:
R_1in0

Matrix([
[cos(psi)*cos(theta), sin(phi)*sin(theta)*cos(psi) - sin(psi)*cos(phi),  sin(phi)*sin(psi) + sin(theta)*cos(phi)*cos(psi)],
[sin(psi)*cos(theta), sin(phi)*sin(psi)*sin(theta) + cos(phi)*cos(psi), -sin(phi)*cos(psi) + sin(psi)*sin(theta)*cos(phi)],
[        -sin(theta),                              sin(phi)*cos(theta),                               cos(phi)*cos(theta)]])

### 2.2.2 Map from angular velocity to angular rates

Recall that

$$\begin{bmatrix} \dot{\psi} \\ \dot{\theta} \\ \dot{\phi} \end{bmatrix} = N w_{0, 1}^{1}$$

for some matrix $N$. Here is how to compute that matrix for a ZYX (yaw, pitch, roll) Euler angle sequence.  First, we compute its inverse:

In [11]:
Ninv = sym.Matrix.hstack((Ry * Rx).T * sym.Matrix([0, 0, 1]),
                              (Rx).T * sym.Matrix([0, 1, 0]),
                                       sym.Matrix([1, 0, 0]))

Then, we compute $N$ by taking the inverse of $N^{-1}$:

In [12]:
N = sym.simplify(Ninv.inv())

In [13]:
N

Matrix([
[0, sin(phi)/cos(theta), cos(phi)/cos(theta)],
[0,            cos(phi),           -sin(phi)],
[1, sin(phi)*tan(theta), cos(phi)*tan(theta)]])

## 2.3 Define equations of motion

This is where this document differs  widely from the standard equations of motion given in the class. The pendulum's affect on the motion of the drone is assumed to be negligible (is this true for the crazyflie? idk, we'll see)

### Deriving the Equations of Motion for the Pendulum

Generalized Co-ordinates: $x$, $z$ (in the world frame), $\theta$ (pitch) and $\alpha$ (angle of the drone from the world $z_0$)

In [14]:
o_x, o_z, alpha, theta = dynamicsymbols(r'o_x o_z \alpha \theta')
o_xd, o_zd, alphad, thetad = dynamicsymbols(r'o_x o_z \alpha \theta', 1)
o_xdd, o_zdd, alphadd, thetadd = dynamicsymbols(r'o_x o_z \alpha \theta', 2)
L_with_z = 1/2*m*(o_xd**2 + o_zd**2) + 1/2*J_y*(thetad**2) + 1/2*m_pen*(o_xd**2 + o_zd**2 + l_pen**2 * alphad**2 + 2*l_pen*alphad*(o_xd*sym.cos(alpha) - o_zd*sym.sin(alpha)) - m*g*o_z - m_pen*g*(o_z + l_pen*sym.cos(alpha)))

LM_with_z = LagrangesMethod(L_with_z, [o_x, o_z, alpha, theta])
mechanics_printing(pretty_print=True)
Equations_with_z = LM_with_z.form_lagranges_equations()
Equations_with_z

⎡                                                                             
⎢                                                             1.0⋅m⋅oₓ̈ + 0.5⋅
⎢                                                                             
⎢                                                                             
⎢                                               1.0⋅m⋅o_̈z - 0.5⋅mₚₑₙ⋅(-g⋅m - 
⎢                                                                             
⎢         ⎛      2                                                            
⎢0.5⋅mₚₑₙ⋅⎝2⋅lₚₑₙ ⋅\alp̈ha + 2⋅lₚₑₙ⋅(-sin(\alpha)⋅\alṗha⋅oₓ̇ - sin(\alpha)⋅o_
⎢                                                                             
⎣                                                                             

    ⎛                           2                                   ⎞         
mₚₑₙ⋅⎝- 2⋅lₚₑₙ⋅sin(\alpha)⋅\alṗha  + 2⋅lₚₑₙ⋅cos(\alpha)⋅\alp̈ha + 2⋅oₓ̈⎠     
                                                   

Solving the kinematic equation for $\ddot{\alpha}$:

In [1]:
alpha_ddot = sym.solve(Equations_with_z[2], alphadd)

NameError: name 'sym' is not defined

Create a list of states, inputs and parameters as symbolic variables.

In [ ]:
s = [o_x, o_y, o_z, psi, theta, phi, v_x, v_y, v_z, w_x, w_y, w_z, alpha, alpha_dot]
i = [tau_x, tau_y, tau_z, f_z]
p = [m, J_x, J_y, J_z, g]

Forces.

In [ ]:
f_in1 = R_1in0.T * sym.Matrix([0, 0, -m * g]) + sym.Matrix([0, 0, f_z])

Torques.

In [ ]:
tau_in1 = sym.Matrix([tau_x, tau_y, tau_z])

In [ ]:
f_in1

Matrix([
[               g*m*sin(theta)],
[     -g*m*sin(phi)*cos(theta)],
[f_z - g*m*cos(phi)*cos(theta)]])

In [ ]:
tau_in1

Matrix([
[tau_x],
[tau_y],
[tau_z]])

Create equations of motion.

In [ ]:
f_sym = sym.Matrix.vstack(R_1in0 * v_01in1,
                          N * w_01in1,
                          (1 / m) * (f_in1 - w_01in1.cross(m * v_01in1)),
                          J_in1.inv() * (tau_in1 - w_01in1.cross(J_in1 * w_01in1)),
                          )

Show equations of motion, which have the form

$$\dot{s} = f(s, i, p)$$

for states

$$
s = \begin{bmatrix} o_x \\ o_y \\ o_z \\ \psi \\ \theta \\ \phi \\ v_x \\ v_y \\ v_z \\ w_x \\ w_y \\ w_z \end{bmatrix},
$$

inputs
$$
i = \begin{bmatrix} \tau_x \\ \tau_y \\ \tau_z \\ f_z \end{bmatrix},
$$

and parameters
$$
p = \begin{bmatrix} m \\ J_x \\ J_y \\ J_z \\ g \end{bmatrix}.
$$

In [ ]:
f_sym

Matrix([
[ v_x*cos(psi)*cos(theta) + v_y*(sin(phi)*sin(theta)*cos(psi) - sin(psi)*cos(phi)) + v_z*(sin(phi)*sin(psi) + sin(theta)*cos(phi)*cos(psi))],
[v_x*sin(psi)*cos(theta) + v_y*(sin(phi)*sin(psi)*sin(theta) + cos(phi)*cos(psi)) + v_z*(-sin(phi)*cos(psi) + sin(psi)*sin(theta)*cos(phi))],
[                                                                       -v_x*sin(theta) + v_y*sin(phi)*cos(theta) + v_z*cos(phi)*cos(theta)],
[                                                                                         w_y*sin(phi)/cos(theta) + w_z*cos(phi)/cos(theta)],
[                                                                                                               w_y*cos(phi) - w_z*sin(phi)],
[                                                                                   w_x + w_y*sin(phi)*tan(theta) + w_z*cos(phi)*tan(theta)],
[                                                                                                (g*m*sin(theta) + m*v_y*w_z - m*v_z*w_y)/m